# 01 - EDA YouTube Toxic Comments

En este notebook realizamos el **Análisis Exploratorio de Datos (EDA)** del dataset de comentarios de YouTube etiquetados por toxicidad.

## Objetivos

- Entender la estructura del dataset (filas, columnas, tipos de datos).
- Analizar la distribución de las diferentes etiquetas de toxicidad (multietiqueta).
- Estudiar la longitud y características básicas de los comentarios.
- Detectar posibles problemas: valores nulos, duplicados, desbalanceo de clases.
- Analizar la relación entre tipos de toxicidad.
- Explorar diferencias entre comentarios tóxicos y no tóxicos.
- Obtener insights que nos ayuden a tomar decisiones de preprocesamiento y modelado.
